In [15]:
%%capture
!pip install pandas openpyxl langchain python-dotenv langchain_openai langchain_experimental tabulate

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
root_file_path = "data/"

file_path = ["2017-aadt-publication.xlsm",
             "2018-AADT-PUBLICATION.xlsx",
             "2019-AADT-PUBLICATION.xlsx",
             "2020-AADT-PUBLICATION.xlsx",
             "2021-AADT-PUBLICATION.xlsx",
             "2022-AADT-PUBLICATION.xlsx",
             "2023-AADT-PUBLICATION.xlsx",]

# Multi DataFrame Method

## Reading CSV

In [18]:
import pandas as pd

final_df = []

for idx, file in enumerate(file_path):
  if idx < len(file_path) - 2:
    df = pd.read_excel(root_file_path + file, skiprows=4, sheet_name='ALL MAINLINE SECTIONS')
  else:
    df = pd.read_excel(root_file_path + file, skiprows=4)
  final_df.append(df)

## Defining LLM

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=0, model="gpt-4o-mini")

## Defining Agent

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

agent = create_pandas_dataframe_agent(
    chat,
    final_df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
    prefix="\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`. Make sure you do every possible thing to answer the user's query. If still you feel there might be some issue with the query, let the user know along with a small solve.",

)

## Evaluating the agent

In [21]:
agent.invoke("What is the AADT from Exit 7 Araby Rd to Exit 3 SR 280 / Avenue 3E in 2021?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# Assuming df is the dataframe containing the data\n\n# Filter for the relevant exits and year 2021\nexit_7_aadt_2021 = df[(df['Start'].str.contains('Exit 7 Araby Rd')) & (df['End'].str.contains('Exit 3 SR 280 / Avenue 3E'))]['AADT 2021']\n\n# Get the AADT value\nexit_7_aadt_2021_value = exit_7_aadt_2021.values\nexit_7_aadt_2021_value"}`
responded: To find the AADT (Annual Average Daily Traffic) from Exit 7 Araby Rd to Exit 3 SR 280 / Avenue 3E in 2021, we need to filter the dataframe for the relevant route and exits, and then extract the AADT for the year 2021.

Let's check the dataframe for the relevant information. The AADT for 2021 is in the column labeled "AADT 2021". 

I'll execute the necessary code to find this information.

NameError: name 'df' is not definedIt seems that I cannot access the dataframe directly in this environment. However, I can guide you on how to do it in your own Python en

{'input': 'What is the AADT from Exit 7 Araby Rd to Exit 3 SR 280 / Avenue 3E in 2021?',
 'output': "It seems that I cannot access the dataframe directly in this environment. However, I can guide you on how to do it in your own Python environment.\n\nYou can use the following code snippet to find the AADT from Exit 7 Araby Rd to Exit 3 SR 280 / Avenue 3E in 2021:\n\n```python\n# Filter for the relevant exits and year 2021\nexit_7_aadt_2021 = df[(df['Start'].str.contains('Exit 7 Araby Rd')) & (df['End'].str.contains('Exit 3 SR 280 / Avenue 3E'))]['AADT 2021']\n\n# Get the AADT value\nexit_7_aadt_2021_value = exit_7_aadt_2021.values\nprint(exit_7_aadt_2021_value)\n```\n\nMake sure to run this code in an environment where the dataframe `df` is defined. If you encounter any issues, please let me know!"}

# Using custom tools

## Reading CSV

In [16]:
import pandas as pd

final_df = []

for idx, file in enumerate(file_path):
  if idx < len(file_path) - 2:
    df = pd.read_excel(root_file_path + file, skiprows=4, sheet_name='ALL MAINLINE SECTIONS')
  else:
    df = pd.read_excel(root_file_path + file, skiprows=4)
  final_df.append(df)

## Defining LLM

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

In [18]:
dataframe_mapping_with_year = {
    "df_2017": final_df[0],
    "df_2018": final_df[1],
    "df_2019": final_df[2],
    "df_2020": final_df[3],
    "df_2021": final_df[4],
    "df_2022": final_df[5],
    "df_2023": final_df[6],
}

In [19]:
def ask_llm_for_file(llm, query):
  prompt = f"You have a list of dataframes. The list contains seven dataframes. Each dataframe contains the AADT data for a year. The dataframes are named as follows: df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023. df_2017 also has information about 2035 Future AADT. df_2018, df_2019, df_2020, df_2021 have information about 2040 Future AADT. df_2022 and df_2023 have information about 2042 and 2043 Future AADT respectively. The dataframes are stored in a list named `final_df`. Given a query, you need to say which dataframe to use to answer the query. The query is: {query}. Which dataframe should be used to answer the query? Only provide the name of the dataframe."

  response = llm.invoke(prompt)
  return response.content
  

In [20]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

def get_agent_for_llm(llm, agent_df):
  agent = create_pandas_dataframe_agent(
      llm,
      agent_df,
      verbose=True,
      agent_type=AgentType.OPENAI_FUNCTIONS,
      allow_dangerous_code=True,
      prefix="\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`. The road names may not be exactly the same. The FromRoad and ToRoad columns contain the road names. Make sure you interchange the road names if needed. If still you feel there might be some issue with the query, let the user know along with a small solve.",
  )

  return agent

# Evaluating the agent

## Query 1

In [21]:
query = "What is the AADT from Exit 7 Araby Rd to Exit 3 SR 280 / Avenue 3E in 2021?"
response = ask_llm_for_file(llm, query)

agent_df = dataframe_mapping_with_year[response]

agent = get_agent_for_llm(llm, agent_df)

output = agent.invoke(query)

output



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# First, let's check the dataframe for the relevant rows\nimport pandas as pd\n\n# Assuming df is already defined\nresult = df[(df['Start'].str.contains('Exit 7 Araby Rd')) & (df['End'].str.contains('Exit 3 SR 280 / Avenue 3E'))]\n\n# If no rows are found, check the reverse as well\nif result.empty:\n    result = df[(df['Start'].str.contains('Exit 3 SR 280 / Avenue 3E')) & (df['End'].str.contains('Exit 7 Araby Rd'))]\n\nresult[['Start', 'End', 'AADT 2021']]"}`


                       Start              End  AADT 2021
3  Exit 3 SR 280 / Avenue 3E  Exit 7 Araby Rd    48563.0The AADT from Exit 7 Araby Rd to Exit 3 SR 280 / Avenue 3E in 2021 is 48,563. 

Note that the road names were interchanged in the query, but the data was correctly retrieved. If you have any further questions or need additional information, feel free to ask!

> Finished chain.


{'input': 'What is the AADT from Exit 7 Araby Rd to Exit 3 SR 280 / Avenue 3E in 2021?',
 'output': 'The AADT from Exit 7 Araby Rd to Exit 3 SR 280 / Avenue 3E in 2021 is 48,563. \n\nNote that the road names were interchanged in the query, but the data was correctly retrieved. If you have any further questions or need additional information, feel free to ask!'}

## Query 2

In [22]:
query = "What is the Positive Direction AADT from Spot Rd to Sentinel Rd in 2023"
response = ask_llm_for_file(llm, query)

agent_df = dataframe_mapping_with_year[response]

agent = get_agent_for_llm(llm, agent_df)

output = agent.invoke(query)

output



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# First, let's check the unique road names in the FromRoad and ToRoad columns to see if we need to interchange them.\nfrom_road = 'Spot Rd'\nto_road = 'Sentinel Rd'\n\n# Check if the roads are in the dataframe\nroad_check = df[(df['FromRoad'].str.contains(from_road, case=False)) | (df['ToRoad'].str.contains(to_road, case=False))]\nroad_check[['FromRoad', 'ToRoad', 'Pos Dir AADT', 'AADT 2023']]"}`


   FromRoad       ToRoad Pos Dir AADT AADT 2023
15  Spot Rd  Sentinel Rd  5798.926452     11655The Positive Direction AADT from Spot Rd to Sentinel Rd in 2023 is approximately **5798.93**.

> Finished chain.


{'input': 'What is the Positive Direction AADT from Spot Rd to Sentinel Rd in 2023',
 'output': 'The Positive Direction AADT from Spot Rd to Sentinel Rd in 2023 is approximately **5798.93**.'}

## Query 3

In [25]:
query = "What is 2040 Future AADT in US191 and Chase creek"
response = ask_llm_for_file(llm, query)

agent_df = dataframe_mapping_with_year[response]

agent = get_agent_for_llm(llm, agent_df)

output = agent.invoke(query)

output



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# First, let's check the dataframe for the relevant road names and their corresponding 2040 Future AADT values.\n\n# Importing pandas to work with the dataframe\nimport pandas as pd\n\n# Assuming df is already defined as the dataframe\n\n# Checking for the relevant rows in the dataframe\nresult = df[(df['FromRoad'].str.contains('US191', case=False, na=False) & df['ToRoad'].str.contains('Chase creek', case=False, na=False)) | \n            (df['FromRoad'].str.contains('Chase creek', case=False, na=False) & df['ToRoad'].str.contains('US191', case=False, na=False))]\n\n# Selecting the relevant columns\nresult_2040_AADT = result[['FromRoad', 'ToRoad', '2040 Future AADT']]\nresult_2040_AADT"}`


KeyError: 'FromRoad'
Invoking: `python_repl_ast` with `{'query': "# Let's check the column names in the dataframe to find the correct ones for road names.\ncolumn_names = df.columns.tolist()\ncolumn_names"}`
respon

{'input': 'What is 2040 Future AADT in US191 and Chase creek',
 'output': 'The unique values in the `Start` and `End` columns show a variety of road names, but it seems that "US191" and "Chase creek" are not present in the expected format. \n\nTo proceed, you can:\n1. Check the unique values for any similar names or variations that might correspond to "US191" or "Chase creek".\n2. If you have specific variations of these road names, please provide them, and I can help you search again.\n\nWould you like to see a sample of the unique values, or do you have specific names in mind that we should check?'}

## Query 4

In [24]:
query = "Single Trucks AADT for 2023 between Circle Mountain and I-17 Front"
response = ask_llm_for_file(llm, query)

agent_df = dataframe_mapping_with_year[response]

agent = get_agent_for_llm(llm, agent_df)

output = agent.invoke(query)

output



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# First, let's check the unique values in the FromRoad and ToRoad columns to see how they relate to the query.\nfrom pandas import DataFrame\n\n# Assuming df is already defined as the DataFrame\nunique_from_roads = df['FromRoad'].unique()\nunique_to_roads = df['ToRoad'].unique()\nunique_from_roads, unique_to_roads"}`


(array(['California State Line - Yuma', 'E Giss Pkwy', 'US-95', ...,
       'E Cordes Lakes Rd WB', '90 ft W of Unidrd Black Canyon',
       'I-17 Exit 244 C-Ramp'], shape=(2359,), dtype=object), array(['E Giss Pkwy', 'US-95', 'SR-280', ..., 'N Curry Rd (A)',
       'E Lichen Ln', 'I-17 Exit 244 A-Ramp'], shape=(2417,), dtype=object))The unique road names in the `FromRoad` and `ToRoad` columns are quite extensive, and it seems that "Circle Mountain" and "I-17 Front" are not directly listed. This could indicate that the road names might be slightly different or that they are represented 

{'input': 'Single Trucks AADT for 2023 between Circle Mountain and I-17 Front',
 'output': 'The unique road names in the `FromRoad` and `ToRoad` columns are quite extensive, and it seems that "Circle Mountain" and "I-17 Front" are not directly listed. This could indicate that the road names might be slightly different or that they are represented in a different format.\n\nTo proceed, I will search for any entries that might correspond to "Circle Mountain" and "I-17 Front" in the `FromRoad` and `ToRoad` columns. If I find any potential matches, I will calculate the AADT for single trucks for 2023 between those roads. If not, I will inform you of the situation.\n\nLet\'s search for potential matches.'}

## Query 5

In [27]:
query = "2020 Negative direction AADT between Marble Canyon and Jacob Lake"
response = ask_llm_for_file(llm, query)

agent_df = dataframe_mapping_with_year[response]

agent = get_agent_for_llm(llm, agent_df)

output = agent.invoke(query)

output



> Entering new AgentExecutor chain...
To find the 2020 Negative direction AADT between Marble Canyon and Jacob Lake, I will first check if the road names are present in the `FromRoad` and `ToRoad` columns of the dataframe. If they are not in the expected order, I will interchange them if necessary. 

Let's proceed with the query.

> Finished chain.


{'input': '2020 Negative direction AADT between Marble Canyon and Jacob Lake',
 'output': "To find the 2020 Negative direction AADT between Marble Canyon and Jacob Lake, I will first check if the road names are present in the `FromRoad` and `ToRoad` columns of the dataframe. If they are not in the expected order, I will interchange them if necessary. \n\nLet's proceed with the query."}

## Query 6

In [28]:
query = "Location ID for Road between Exit 126 Estrella Pk and Exit 128 Bullard Av"
response = ask_llm_for_file(llm, query)

agent_df = dataframe_mapping_with_year[response]

agent = get_agent_for_llm(llm, agent_df)

output = agent.invoke(query)

output



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# First, let's find the rows that contain the specified road names in the Start and End columns.\n\n# Define the road names\nfrom_road = 'Exit 126 Estrella Pk'\nto_road = 'Exit 128 Bullard Av'\n\n# Check for the rows in the dataframe\nresult = df[(df['Start'].str.contains(from_road)) & (df['End'].str.contains(to_road)) | (df['Start'].str.contains(to_road)) & (df['End'].str.contains(from_road))]\n\n# Get the Loc ID if any results are found\nloc_id = result['Loc ID'].tolist()\nloc_id"}`


[]It seems that there are no exact matches for the roads "Exit 126 Estrella Pk" and "Exit 128 Bullard Av" in the dataframe. This could be due to slight variations in the road names or the way they are recorded.

To assist you better, please check the following:
1. Ensure that the road names are spelled correctly.
2. Consider any alternative names or abbreviations that might be used in the dataframe.

If you have any ot

{'input': 'Location ID for Road between Exit 126 Estrella Pk and Exit 128 Bullard Av',
 'output': 'It seems that there are no exact matches for the roads "Exit 126 Estrella Pk" and "Exit 128 Bullard Av" in the dataframe. This could be due to slight variations in the road names or the way they are recorded.\n\nTo assist you better, please check the following:\n1. Ensure that the road names are spelled correctly.\n2. Consider any alternative names or abbreviations that might be used in the dataframe.\n\nIf you have any other road names or additional details, please provide them, and I can help you search again!'}

## Query 7

In [29]:
query = "Provide me the Start and End names of road with the highest AADT in 2023"
response = ask_llm_for_file(llm, query)

agent_df = dataframe_mapping_with_year[response]

agent = get_agent_for_llm(llm, agent_df)

output = agent.invoke(query)

output



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# Find the row with the highest AADT in 2023\nhighest_aadt_row = df.loc[df['AADT 2023'].idxmax()]\n\n# Get the FromRoad and ToRoad\nfrom_road = highest_aadt_row['FromRoad']\nto_road = highest_aadt_row['ToRoad']\n\n# Check if FromRoad and ToRoad need to be interchanged\nif from_road > to_road:\n    from_road, to_road = to_road, from_road\n\nfrom_road, to_road"}`


TypeError: '>' not supported between instances of 'str' and 'int'
Invoking: `python_repl_ast` with `{'query': "# Find the row with the highest AADT in 2023\nhighest_aadt_row = df.loc[df['AADT 2023'].idxmax()]\n\n# Get the FromRoad and ToRoad\nfrom_road = highest_aadt_row['FromRoad']\nto_road = highest_aadt_row['ToRoad']\n\n# Check if FromRoad and ToRoad need to be interchanged\nif from_road < to_road:\n    from_road, to_road = to_road, from_road\n\nfrom_road, to_road"}`


TypeError: '>' not supported between instances of 'str' and 'int'
Invok

{'input': 'Provide me the Start and End names of road with the highest AADT in 2023',
 'output': 'The road names with the highest AADT in 2023 are:\n\n- From Road: **7th St**\n- To Road: **16th St**\n\nThese names have been interchanged to ensure they are in the correct order. If you have any further questions or need additional assistance, feel free to ask!'}